In [13]:
from hs_gimme.logging_service.problems_only_logging_service import ProblemsOnlyLoggingService
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from hs_recommendations_api_client import get_recommendations_api_client
from tqdm import tqdm
from json2html import *
from hs_gimme.s3.resume_storage import ResumeStorage
from hs_gimme.hs_object_functions.application import get_selected_resume
from hs_gimme.constants.objects.application import ResumeData
import logging
import pandas as pd
import itertools
from IPython.core.display import display, HTML

/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_76045/3907335583.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
ENV = 'production'
ACCOUNT = 'jaipur'
request_kwargs = {"fetch_mode": {"value": "employee_fetch"},
        "allow_already_applied_req": {"value": True},
        "allow_candidates_with_offer": {"value": True}}

In [3]:
mongo = get_mongo_client_db(ENV, ACCOUNT)

In [4]:
logger = logging.getLogger()

In [5]:
resume_storage = ResumeStorage(logger)

In [ ]:
req_ids = mongo.req.distinct('_id', {'req_status': 'Open'})
len(req_ids)

In [6]:
df = pd.read_excel(f'/Users/dima/Downloads/for_mer/{ACCOUNT}_{ENV}.xlsx')
len(df)

109

In [7]:
req_ids = list(df['req_id'].unique())
len(req_ids)

52

In [8]:
api = get_recommendations_api_client(environment=ENV, logger=ProblemsOnlyLoggingService())

In [9]:
recommandations = {}

for req_id in tqdm(req_ids[:200]):
    r = api.fast_fetch(ACCOUNT, req_id, **request_kwargs)
    #print(f'{i}/{len(req_ids)}', req_id, r['total_count'])
    if r['total_count'] > 0:
        recommandations[req_id] = r
len(recommandations)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [00:37<00:00,  1.38it/s]


10

In [14]:
len(list(itertools.chain(*recommandations.values())))

40

In [ ]:
def add_html_paragraph(text):
    return '<p>' + text + '</p>'

def load_resume_bytes(app_from_mongo: str,  _resume_storage: ResumeStorage):
    resume_data = get_selected_resume(app_from_mongo)
    unique_key = resume_data.get(ResumeData.PDF_UNIQUE_KEY) or resume_data[ResumeData.UNIQUE_KEY]
    resume_bytes = _resume_storage.get_resume_bytes(resume_data, unique_key=unique_key)
    return resume_bytes

In [25]:
data = []
for req_id, recomendation in recommandations.items():
    print(req_id, len(recomendation['results']))
    for result in recomendation['results']:
        
        candidate = mongo.application.find_one(result['candidate_data']['last_application_id'])
        candidate_id = candidate['_id']
        
#         employee = mongo.employee.find_one({'current_talent_id': result['candidate_data']['current_talent_id']})
#         req = mongo.req.find_one(req_id)
#         req_html = employee_html = """<html><style>
#                         .styled-table {
#                     border-collapse: collapse;
#                     margin: 25px 0;
#                     font-size: 0.9em;
#                     font-family: sans-serif;
#                     min-width: 400px;
#                     box-shadow: 0 0 20px rgba(0, 0, 0, 0.15);
#                 }
                
#                 th {
#                     vertical-align: top;
#                 }
#             </style>
#             <body>
#             """
#         req_html += f'<h1>Req {req_id}:</h1>'
#         req_summary = { k: req.get(k) for k in ['city', 'address', 'country', 'job_title', 'job_department',
#                                              'top_category', 'visa_requirements'] }
#         req_html += add_html_paragraph(json2html.convert(req_summary, table_attributes='class=styled-table'))

        
#         for k, t in req['job_listing'].items():
#             req_html += '<h3>' + k + '</h3>'
#             req_html += add_html_paragraph(t)
            
#         employee_html += f'<h1>Emplyee {candidate_id}:</h1>'
        
#         employee_summary = { k: employee.get(k) for k in ['contact_info', 'current_position', 'experience', 'ratings', 'skills',
#                                               'hcm_samurai'] }
        
#         employee_html += '<h2>Data from HCM:</h2>'
#         employee_html += add_html_paragraph(json2html.convert(employee_summary, table_attributes='class=styled-table'))
        
        
# #         html += '<h2>Last Application resume:</h2>'
# #         html += add_html_paragraph(candidate['samurai_json']['resume_as_text'].replace('\n', '<br>'))
        
# #         html == '<h2>Insights:</h2>'
# #         html += add_html_paragraph(json2html.convert(result['insights'], table_attributes='class=styled-table'))
        
# #         html += '<h2>Boosts:</h2>'
# #         html += add_html_paragraph(json2html.convert(result['boosts'], table_attributes='class=styled-table'))
        
# #         html += '<h2>Grading Data:</h2>'
# #         html += add_html_paragraph(json2html.convert(result['grading_data'], table_attributes='class=styled-table'))
        
#         req_html += '</body>'
#         employee_html += '</body>'
        
        
#         req_path = f'/Users/dima/Downloads/for_mer/utah/{req_id}.htm'
#         with open(req_path, 'w') as f:
#             f.write(req_html)
#             print(req_path)
            
#         emplyee_path = f'/Users/dima/Downloads/for_mer/utah/{candidate_id}.htm'
#         with open(emplyee_path, 'w') as f:
#             f.write(employee_html)
#             print(emplyee_path)
            
            
#         resume_bytes = load_resume_bytes(candidate, resume_storage)

#         resume_path = f'/Users/dima/Downloads/for_mer/utah/{candidate_id}.pdf'
#         with open(resume_path, 'wb') as f:
#             print(resume_path)
#             f.write(resume_bytes)
#         req_path = f'/Users/dima/Downloads/for_mer/utah/{req_id}.htm'
#         with open(req_path, 'w') as f:
#             f.write(req_html)
#             print(req_path)
        
        d = {
                'req_id': req_id,
                'candidate_id': candidate_id
            }

        for f in ['grade', 'filtering_score']:
            d[f] = result[f]

        d['recommendation'] = result
        data.append(d)

JLLREQ177536 2
JLLREQ177552 1
JLLREQ213226 3
JLLREQ225478 1
JLLREQ227197 1
JLLREQ234115 1
JLLREQ234127 1
JLLREQ237680 1
JLLREQ238357 1
JLLREQ238385 4


In [22]:
df = pd.DataFrame(data)
# df.to_excel(f'/Users/dima/Downloads/for_mer/utah/{ACCOUNT}_{ENV}.xlsx')
# df.head()

In [23]:
len(df)

16

In [16]:
recomendation['results'][0]

{'candidate_id': 'JLLCAND2180263',
 'candidate_data': {'email': 'progers_2001@yahoo.com',
  'name': 'Phillip Rogers Jr',
  'logical_update_date': '2021-06-01 18:16:00.172000',
  'last_applied_date': '2021-06-01 18:16:00.172000',
  'highest_degree': 'Bachelors',
  'last_title': 'AEROTEK',
  'last_application_id': 'JLLREQ145975_JLLCAND2180263',
  'latest_entity_id': 'JLLREQ145975_JLLCAND2180263',
  'latest_entity_type': 'ats',
  'latest_entity_source_system': 'workday',
  'current_talent_id': '272c3515-3783-4434-83f8-57eb6fd528e0',
  'talent_ids': ['272c3515-3783-4434-83f8-57eb6fd528e0'],
  'data_sources': ['hcm', 'ats'],
  'is_internal': True,
  'high_prob_to_apply': False,
  'prob_to_apply': 0.004,
  'applied_req_ids_for_match': ['JLLREQ147817', 'JLLREQ88724'],
  'max_past_phase': 'ready_for_hire',
  'last_company': 'Roche Diagnostics',
  'population_types': [],
  'source_systems': ['hcm', 'workday'],
  'tags': [],
  'external_pool': {'account_id': 'jaipur',
   'external_talent_id': '2